In [518]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.cluster import DBSCAN
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

This set of features got a silhouette score of ~.72. The second features list beats this, but saving this here for the moment.

`features = ['subway_count_tenth_mi',
       'subway_count_half_mi', 'subway_count_one_mi',
       'subway_count_two_five_mi',
       'park_count_tenth_mi', 'park_count_half_mi', 'park_count_one_mi',
       'park_count_two_five_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND',
       'owners_business_type_CORPORATION', 'owners_business_type_HPD',
       'owners_business_type_INDIVIDUAL', 'owners_business_type_NYC AGENCY',
       'owners_business_type_NYCHA/HHC', 'owners_business_type_OTHER',
       "owners_business_type_OTHER GOV'T AGENCY",
       'owners_business_type_PARTNERSHIP', 'non-profit_Y', 'in_hist_dist_1']`

# Importing Data and Test Data

In [319]:
permits = pd.read_csv('data/final_permits.csv')

In [538]:
test = pd.read_csv('data/test_data.csv')

# Pairplot
It would be great to make one, but there are too many features

# DBSCAN with First Feature Set

## Building First Feature Set

In [320]:
permits.columns

Index(['bin_no', 'house_no', 'street_name', 'job_no', 'zip_code',
       'job_start_date', 'latitude', 'longitude', 'total_complaints',
       'closest_subway',
       ...
       'bct2020_5027702', 'bct2020_5027704', 'bct2020_5027900',
       'bct2020_5029102', 'bct2020_5029104', 'bct2020_5029105',
       'bct2020_5030302', 'bct2020_5031901', 'bct2020_5031902',
       'bct2020_5032300'],
      dtype='object', length=1451)

In [321]:
permits.shape

(7177, 1451)

In [322]:
col_list = permits.columns.to_list()

In [323]:
col_list

['bin_no',
 'house_no',
 'street_name',
 'job_no',
 'zip_code',
 'job_start_date',
 'latitude',
 'longitude',
 'total_complaints',
 'closest_subway',
 'closest_subway_line',
 'subway_count_tenth_mi',
 'subway_count_half_mi',
 'subway_count_one_mi',
 'subway_count_two_five_mi',
 'hist_dist_name',
 'closest_park',
 'name_closest_park',
 'park_count_tenth_mi',
 'park_count_half_mi',
 'park_count_one_mi',
 'park_count_two_five_mi',
 'pop_20',
 'hhpop_20p',
 'gq_20p',
 'instgq_20p',
 'avhhsz_20',
 'popu18_20p',
 'hsp_20p',
 'wnh_20p',
 'bnh_20p',
 'anh_20p',
 'onh_20p',
 'nh2pl_20p',
 'hunits_20',
 'ochu_20p',
 'vachu_20p',
 'borough_BROOKLYN',
 'borough_MANHATTAN',
 'borough_QUEENS',
 'borough_STATEN ISLAND',
 'owners_business_type_CORPORATION',
 'owners_business_type_HPD',
 'owners_business_type_INDIVIDUAL',
 'owners_business_type_NYC AGENCY',
 'owners_business_type_NYCHA/HHC',
 'owners_business_type_OTHER',
 "owners_business_type_OTHER GOV'T AGENCY",
 'owners_business_type_PARTNERSHIP',


In [375]:
to_drop1 = ['bin_no', 'house_no', 'street_name','zip_code', 'job_start_date','hist_dist_name','name_closest_park', 'closest_subway_line']

In [376]:
to_drop1

['bin_no',
 'house_no',
 'street_name',
 'zip_code',
 'job_start_date',
 'hist_dist_name',
 'name_closest_park',
 'closest_subway_line']

In [377]:
permits.shape

(7177, 1451)

In [378]:
perms_use1 = permits.drop(columns = to_drop1)

In [379]:
perms_use1.head()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1,bct2020_1001200,bct2020_1001402,bct2020_1001501,bct2020_1001502,bct2020_1001600,bct2020_1001800,bct2020_1002100,bct2020_1002201,bct2020_1002202,bct2020_1002601,bct2020_1002602,bct2020_1002800,bct2020_1003001,bct2020_1003002,bct2020_1003300,bct2020_1003400,bct2020_1003601,bct2020_1003602,bct2020_1003700,bct2020_1003800,bct2020_1003900,bct2020_1004100,bct2020_1004200,bct2020_1004300,bct2020_1004500,bct2020_1004700,bct2020_1005400,bct2020_1005502,bct2020_1005800,bct2020_1006400,bct2020_1006800,bct2020_1006900,bct2020_1007002,bct2020_1007100,bct2020_1007300,bct2020_1007400,bct2020_1007500,bct2020_1007800,bct2020_1008100,bct2020_1008700,bct2020_1008800,bct2020_1008900,bct2020_1009100,bct2020_1009500,bct2020_1009700,bct2020_1009800,bct2020_1009901,bct2020_1009902,bct2020_1009903,bct2020_1010000,bct2020_1011100,bct2020_1011201,bct2020_1011500,bct2020_1012101,bct2020_1012700,bct2020_1012901,bct2020_1012902,bct2020_1013201,bct2020_1013300,bct2020_1013501,bct2020_1013502,bct2020_1013602,bct2020_1013800,bct2020_1013900,bct2020_1014000,bct2020_1014200,bct2020_1014402,bct2020_1014500,bct2020_1014601,bct2020_1014801,bct2020_1014802,bct2020_1014900,bct2020_1015001,bct2020_1015002,bct2020_1015102,bct2020_1015200,bct2020_1015402,bct2020_1015501,bct2020_1015601,bct2020_1015900,bct2020_1016200,bct2020_1016400,bct2020_1016600,bct2020_1016700,bct2020_1016900,bct2020_1017000,bct2020_1017200,bct2020_1017401,bct2020_1017402,bct2020_1017500,bct2020_1017800,bct2020_1017900,bct2020_1018000,bct2020_1018100,bct2020_1018200,bct2020_1018300,bct2020_1018600,bct2020_1018700,bct2020_1018800,bct2020_1018900,bct2020_1019000,bct2020_1019100,bct2020_1019200,bct2020_1019300,bct2020_1019400,bct2020_1019600,bct2020_1019702,bct2020_1019800,bct2020_1020000,bct2020_1020102,bct2020_1020500,bct2020_1020600,bct2020_1020701,bct2020_1020800,bct2020_1020901,bct2020_1021100,bct2020_1021303,bct2020_1021500,bct2020_1021800,bct2020_1021900,bct2020_1022102,bct2020_1022200,bct2020_1022400,bct2020_1022600,bct2020_1022700,bct2020_1022800,bct2020_1023000,bct2020_1023100,bct2020_1023200,bct2020_1023300,bct2020_1023400,bct2020_1023501,bct2020_1023502,bct2020_1023600,bct2020_1023700,bct2020_1023804,bct2020_1024200,bct2020_1024301,bct2020_1024500,bct2020_1025100,bct2020_1025700,bct2020_1025900,bct2020_1026900,bct2020_1027300,bct2020_1027900,bct2020_1028300,bct2020_1028700,bct2020_1029100,bct2020_1029500,bct2020_2000200,bct2020_2000400,bct2020_2001600,bct2020_2001901,bct2020_2001902,bct2020_2002001,bct2020_2002002,bct2020_2002500,bct2020_2002701,bct2020_2002800,bct2020_2003100,bct2020_2003300,bct2020_2003500,bct2020_2003800,bct2020_2004100,bct2020_2004300,bct2020_2004400,bct2020_2005002,bct2020_2005100,bct2020_2005902,bct2020_2006301,bct2020_2006400,bct2020_2006500,bct2020_2006700,bct2020_2006900,bct2020_2007000,bct2020_2007100,bct2020_2007300,bct2020_2007500,bct2020_2007600,bct2020_2007700,bct2020_2007800,bct2020_2007900,bct2020_2008700,bct2020_2008900,bct2020_2009000,bct2020_2009200,bct2020_2009301,bct2020_2009600,bct2020_2011502,bct2020_2011800,bct2020_2012101,bct2020_2012102,bct2020_2012300,bct2020_2012500,bct2020_2012701,bct2020_2012901,bct2020_2013000,bct2020_2013100,bct2020_2013200,bct2020_2013300,bct2020_2013500,bct2020_2013800,bct2020_2014100,bct2020_2014300,bct2020_

In [380]:
perms_use1.shape

(7177, 1443)

## DBSCAN
I wrote this for loop as an alternative to GridSearchCV, which doesn't have a scorer for DBSCAN. If it's possible to use `make_scorer`, it quickly became clear it would be more complicated than writing a for loop, so I just did that.

In building this for loop to return the keys that produce the best silhouette scores, I opted to build a dictionary of `{'eps': 'silhouette_score'}`. These pages helped me to pull out the best epsilons:
- [This page](https://www.geeksforgeeks.org/python-get-key-from-value-in-dictionary/) showed me how to find the key based on the value.
- [This page](https://www.delftstack.com/howto/python/python-max-value-in-list/) reminded me of the `max()` function.

It appears 9 through 11 are equal, and produce the best silhouette score.

Having played with fewer features previously and getting a higher silhouette score, I'm going to try eliminating features and see what happens.

In [381]:
#to track best scores
best_scores = {}
best_eps = {}

In [548]:
best_params = {}

In [382]:
X = perms_use1
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [383]:
features = 'features1' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,16):
    dbscan = DBSCAN(eps = num)
    dbscan.fit(X_scaled)
    score = silhouette_score(X_scaled, dbscan.labels_)
    pairs[num] = score
    scores.append(score)
    clusters = len(pd.Series(dbscan.labels_).unique())
    print(f"For epsilon = {num}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 1, the silhouette_score is 0.09734756904148606 and there are 261 clusters
For epsilon = 2, the silhouette_score is 0.18292111673595113 and there are 313 clusters
For epsilon = 3, the silhouette_score is 0.29232976121750265 and there are 345 clusters
For epsilon = 4, the silhouette_score is 0.5256902929137033 and there are 425 clusters
For epsilon = 5, the silhouette_score is 0.5367543596810687 and there are 431 clusters
For epsilon = 6, the silhouette_score is 0.5406652920943468 and there are 433 clusters
For epsilon = 7, the silhouette_score is 0.5595970003620722 and there are 444 clusters
For epsilon = 8, the silhouette_score is 0.5629742736682358 and there are 442 clusters
For epsilon = 9, the silhouette_score is 0.5631723159341483 and there are 442 clusters
For epsilon = 10, the silhouette_score is 0.5631723159341483 and there are 442 clusters
For epsilon = 11, the silhouette_score is 0.5631723159341483 and there are 442 clusters
For epsilon = 12, the silhouette_score

# DBSCAN with Second Feature Set

## Building Second Feature Set

In [384]:
permits_nobct2020 = pd.read_csv('data/final_permits_nobct2020.csv')

In [385]:
permits_nobct2020.shape

(7177, 52)

In [386]:
to_drop2 = to_drop1 + ['bct2020']
to_drop2

['bin_no',
 'house_no',
 'street_name',
 'zip_code',
 'job_start_date',
 'hist_dist_name',
 'name_closest_park',
 'closest_subway_line',
 'bct2020']

In [387]:
perms_use2 = permits_nobct2020.drop(columns = to_drop2)
perms_use2.shape

(7177, 43)

## DBSCAN

In [388]:
X = perms_use2
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

The silhouette score was still improving at 15 when I did range(1,16)' so I increased it to `range(15,31)`. That still produced an increasing silhouette score, so I continued increasing the epsilons I tested `range(29,46)`. At that point, the silhouette score stopped improving (all the scores were identical at 0.872)

In [389]:
features = 'features2' #CHANGE THIS
pairs = {}
scores = []
for num in range(15, 30):
    dbscan = DBSCAN(eps = num)
    dbscan.fit(X_scaled)
    score = silhouette_score(X_scaled, dbscan.labels_)
    pairs[num] = score
    scores.append(score)
    clusters = len(pd.Series(dbscan.labels_).unique())
    print(f"For epsilon = {num}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 15, the silhouette_score is 0.5986355401028285 and there are 4 clusters
For epsilon = 16, the silhouette_score is 0.5986355401028285 and there are 4 clusters
For epsilon = 17, the silhouette_score is 0.5986355401028285 and there are 4 clusters
For epsilon = 18, the silhouette_score is 0.5986355401028285 and there are 4 clusters
For epsilon = 19, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 20, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 21, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 22, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 23, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 24, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 25, the silhouette_score is 0.7204940980071437 and there are 3 clusters
For epsilon = 26, the silhouette_score is 0.7204940980

In [390]:
best_scores

{'features1': 0.5631723159341483, 'features2': 0.8598814471096117}

In [391]:
best_eps

{'features1': [9, 10, 11], 'features2': [29]}

-----

## Examining Features 2 With 3 Clusters (including -1)

In [392]:
dbscan = DBSCAN(eps = 29)
dbscan.fit(X_scaled)
silhouette_score(X_scaled, dbscan.labels_)

0.8598814471096117

In [393]:
pd.Series(dbscan.labels_).unique()

array([ 0, -1])

I got this approach to finding the count of each label at [this stackoverflow answer](https://stackoverflow.com/a/28663910/5394724). There are only 5 outliers here. -1 represents outliers ([DBSCAN documentation](https://scikit-learn.org/stable/modules/clustering.html#dbscan)).

In [394]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{-1: 4, 0: 7173}

In [395]:
permits_X = X
permits_X['cluster']=dbscan.labels_
permits_X.head()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1,cluster
0,121207354,40.798817,-73.968740,2255,0.033418,5,15,42,195,0.000000,2,17,33,214,9005,0.963,0.037,0.0,1.89,0.129,0.159,0.614,0.081,0.099,0.008,0.040,4951,0.924,0.076,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,121189524,40.860296,-73.926125,701,0.085621,2,15,34,153,0.049425,1,10,42,280,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,121205445,40.858508,-73.929532,546,0.177308,0,6,35,146,0.115871,0,11,41,275,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,220586168,40.818565,-73.918118,1578,0.167270,0,9,60,215,0.086285,1,32,93,418,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,220593463,40.818742,-73.915997,2368,0.196656,0,12,61,216,0.134117,0,32,90,422,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [396]:
pd.set_option('display.max_columns', None)

In [397]:
permits_X.groupby(['cluster']).mean()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,2.957216e+08,40.722350,-73.936880,788.000000,0.349939,0.000000,6.000000,40.500000,231.750000,0.069479,1.500000,31.500000,85.000000,344.75000,2263.250000,0.994500,0.005500,0.000000,2.347500,0.185750,0.281500,0.168750,0.470250,0.032500,0.010250,0.037000,1000.500000,0.953750,0.046250,0.75000,0.000000,0.00000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000
0,3.856172e+08,40.674845,-73.960096,349.456713,0.852850,0.223198,5.224871,18.071518,94.483061,0.152404,0.723686,9.770389,30.836331,145.68242,4409.913147,0.980751,0.019249,0.005124,2.773081,0.206479,0.240441,0.390679,0.182024,0.141521,0.014374,0.030939,1739.317859,0.930115,0.069885,0.30573,0.039035,0.28914,0.264325,0.420884,0.0,0.342674,0.018681,0.002928,0.001255,0.0,0.212463,0.026628,0.005855


Of the four outliers, three (job_nos 320596262, 320913829, 320913785) are basically at the same location, in Brooklyn. The last (job_no 220462381) is in the Bronx. 

I confirmed these by searching the longitude and latitude in Google maps given that I had found a point that was in Albany left in the data set.

In [399]:
permits_X[permits_X['cluster'] == -1]

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1,cluster
4956,320596262,40.690684,-73.944259,673,0.410260,0,4,33,240,0.034073,2,31,84,320,2088,0.993,0.007,0.0,2.30,0.167,0.186,0.217,0.507,0.036,0.011,0.043,956,0.942,0.058,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1
4957,320913829,40.690670,-73.944382,677,0.403379,0,4,33,240,0.039769,2,31,84,321,2088,0.993,0.007,0.0,2.30,0.167,0.186,0.217,0.507,0.036,0.011,0.043,956,0.942,0.058,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1
4958,320913785,40.690676,-73.944321,669,0.406783,0,4,33,240,0.036877,2,31,84,320,2088,0.993,0.007,0.0,2.30,0.167,0.186,0.217,0.507,0.036,0.011,0.043,956,0.942,0.058,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1
5670,220462381,40.817371,-73.914557,1133,0.179334,0,12,63,207,0.167197,0,33,88,418,2789,0.999,0.001,0.0,2.49,0.242,0.568,0.024,0.360,0.022,0.008,0.019,1134,0.989,0.011,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1


# Examining Features 2 With 3 Clusters (including -1)

The outliers are the same as for 2 clusters. Cluster 1 (9 members) is in the Bronx. I don't examine further here because there are other models that are more important to the analysis.

In [400]:
dbscan = DBSCAN(eps = 19)
dbscan.fit(X_scaled)
silhouette_score(X_scaled, dbscan.labels_)

0.7204940980071437

In [401]:
pd.Series(dbscan.labels_).unique()

array([ 0,  1, -1])

In [402]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{-1: 4, 0: 7164, 1: 9}

In [403]:
permits_X = X
permits_X['cluster']=dbscan.labels_
permits_X.head()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1,cluster
0,121207354,40.798817,-73.968740,2255,0.033418,5,15,42,195,0.000000,2,17,33,214,9005,0.963,0.037,0.0,1.89,0.129,0.159,0.614,0.081,0.099,0.008,0.040,4951,0.924,0.076,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
1,121189524,40.860296,-73.926125,701,0.085621,2,15,34,153,0.049425,1,10,42,280,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
2,121205445,40.858508,-73.929532,546,0.177308,0,6,35,146,0.115871,0,11,41,275,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
3,220586168,40.818565,-73.918118,1578,0.167270,0,9,60,215,0.086285,1,32,93,418,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,220593463,40.818742,-73.915997,2368,0.196656,0,12,61,216,0.134117,0,32,90,422,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [404]:
permits_X.groupby(['cluster']).mean()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,2.957216e+08,40.722350,-73.936880,788.000000,0.349939,0.000000,6.000000,40.500000,231.750000,0.069479,1.500000,31.500000,85.000000,344.750000,2263.250000,0.994500,0.005500,0.000000,2.347500,0.185750,0.281500,0.168750,0.470250,0.032500,0.010250,0.037000,1000.500000,0.953750,0.046250,0.750000,0.000000,0.000000,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.0,0.5,0.00000,0.000000,0.000000
0,3.857681e+08,40.674787,-73.960102,348.620882,0.853720,0.221803,5.214126,18.030430,94.203518,0.152485,0.723618,9.758515,30.796622,145.484087,4408.841011,0.980759,0.019241,0.005129,2.773876,0.206574,0.240510,0.390617,0.182092,0.141455,0.014379,0.030926,1737.993579,0.930138,0.069862,0.305416,0.038666,0.289363,0.264657,0.421413,0.0,0.343104,0.018705,0.002931,0.0,0.0,0.21273,0.026521,0.005863
1,2.655426e+08,40.721677,-73.955437,1014.777778,0.160867,1.333333,13.777778,50.777778,317.000000,0.087729,0.777778,19.222222,62.444444,303.555556,5263.333333,0.974111,0.025889,0.001333,2.140000,0.130333,0.185333,0.440333,0.128444,0.194000,0.010778,0.041222,2793.444444,0.911778,0.088222,0.555556,0.333333,0.111111,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.00000,0.111111,0.000000


In [405]:
permits_X[permits_X['cluster'] == -1]

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,owners_business_type_CORPORATION,owners_business_type_HPD,owners_business_type_INDIVIDUAL,owners_business_type_NYC AGENCY,owners_business_type_NYCHA/HHC,owners_business_type_OTHER,owners_business_type_OTHER GOV'T AGENCY,owners_business_type_PARTNERSHIP,non-profit_Y,in_hist_dist_1,cluster
4956,320596262,40.690684,-73.944259,673,0.410260,0,4,33,240,0.034073,2,31,84,320,2088,0.993,0.007,0.0,2.30,0.167,0.186,0.217,0.507,0.036,0.011,0.043,956,0.942,0.058,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1
4957,320913829,40.690670,-73.944382,677,0.403379,0,4,33,240,0.039769,2,31,84,321,2088,0.993,0.007,0.0,2.30,0.167,0.186,0.217,0.507,0.036,0.011,0.043,956,0.942,0.058,1,0,0,0,0,0,0,0,0,0,1,0,0,0,-1
4958,320913785,40.690676,-73.944321,669,0.406783,0,4,33,240,0.036877,2,31,84,320,2088,0.993,0.007,0.0,2.30,0.167,0.186,0.217,0.507,0.036,0.011,0.043,956,0.942,0.058,1,0,0,0,0,1,0,0,0,0,0,0,0,0,-1
5670,220462381,40.817371,-73.914557,1133,0.179334,0,12,63,207,0.167197,0,33,88,418,2789,0.999,0.001,0.0,2.49,0.242,0.568,0.024,0.360,0.022,0.008,0.019,1134,0.989,0.011,0,0,0,0,0,0,0,0,0,0,1,0,0,0,-1


# Tested Additional Feature Set
I made one small change -- I realized that the ochu_20p and vachu_20p are totally correlated so I'm dropping vachu_20p to see what that does. I found no change from Feature Set 2, so I've removed that work from the notebook.

# Function To See If New Data Fits Clusters
Using code from [this stackoverflow answer](https://stackoverflow.com/a/35458920/5394724) to see if my new points fit in these clusters or not.

This compares the new element to the core.

In [543]:
def dbscan_predict(dbscan_model, X_new, metric=sp.spatial.distance.cosine):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        for i, x_core in enumerate(dbscan_model.components_): 
            if metric(x_new, x_core) < dbscan_model.eps:
                # Assign label of x_core to x_new
                y_new[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
                break

    return y_new

# Features 3 - Building Model for Testing
There are some interesting features I don't have data for my test points (who owns, is it a nonprofit?)
I'm repeating the above with those features removed.

After eliminating problematic data I uncovered (2 points outside NYC that sleeped through), the max epsilon that leaves 1 cluster plus outliers is 13, with 2 clusters starting at epsilon = 11.

In [406]:
to_drop3 = to_drop2 + ['owners_business_type', 'non-profit']
to_drop3

['bin_no',
 'house_no',
 'street_name',
 'zip_code',
 'job_start_date',
 'hist_dist_name',
 'name_closest_park',
 'closest_subway_line',
 'bct2020',
 'owners_business_type',
 'non-profit']

In [407]:
permits_for_testing = pd.read_csv('data/final_permits_for_testing.csv')

In [408]:
permits_for_testing.head()

,bin_no,house_no,street_name,job_no,zip_code,job_start_date,owners_business_type,non-profit,latitude,longitude,bct2020,total_complaints,closest_subway,closest_subway_line,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,hist_dist_name,closest_park,name_closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
0,1056547,2686,BROADWAY,121207354,10025,2022-05-11,PARTNERSHIP,N,40.798817,-73.968740,1019100,2255,0.033418,1,5,15,42,195,none,0.000000,Broadway Malls,2,17,33,214,9005,0.963,0.037,0.0,1.89,0.129,0.159,0.614,0.081,0.099,0.008,0.040,4951,0.924,0.076,0,1,0,0,0
1,1812187,140,HILLSIDE AVENUE,121189524,10040,2022-05-11,CORPORATION,N,40.860296,-73.926125,1028300,701,0.085621,1,2,15,34,153,none,0.049425,Highbridge Park,1,10,42,280,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0
2,1091669,37,HILLSIDE AVENUE,121205445,10040,2021-02-19,PARTNERSHIP,N,40.858508,-73.929532,1028300,546,0.177308,A,0,6,35,146,none,0.115871,Fort Tryon Park,0,11,41,275,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0
3,2823631,368,EAST 152 STREET,220586168,10455,2022-05-11,INDIVIDUAL,N,40.818565,-73.918118,2006700,1578,0.167270,2-5,0,9,60,215,none,0.086285,Melrose Playground,1,32,93,418,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0
4,2129278,418,EAST 153 STREET,220593463,10455,2018-02-09,CORPORATION,N,40.818742,-73.915997,2006700,2368,0.196656,2-5,0,12,61,216,none,0.134117,P.S. 29 Ballfield,0,32,90,422,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0


In [409]:
permits_for_testing.shape

(7177, 45)

In [410]:
perms_use3 = permits_for_testing.drop(columns = to_drop3)
perms_use3.shape

(7177, 34)

In [411]:
X = perms_use3
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [413]:
features = 'features3' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,14):
    dbscan = DBSCAN(eps = num)
    dbscan.fit(X_scaled)
    score = silhouette_score(X_scaled, dbscan.labels_)
    pairs[num] = score
    scores.append(score)
    clusters = len(pd.Series(dbscan.labels_).unique())
    print(f"For epsilon = {num}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 1, the silhouette_score is 0.06083164853386988 and there are 298 clusters
For epsilon = 2, the silhouette_score is 0.03276272647072038 and there are 159 clusters
For epsilon = 3, the silhouette_score is 0.0919812858890746 and there are 48 clusters
For epsilon = 4, the silhouette_score is 0.115483871852087 and there are 20 clusters
For epsilon = 5, the silhouette_score is 0.3331186548247646 and there are 10 clusters
For epsilon = 6, the silhouette_score is 0.3316988628640024 and there are 7 clusters
For epsilon = 7, the silhouette_score is 0.42859658731849254 and there are 5 clusters
For epsilon = 8, the silhouette_score is 0.4297740365196608 and there are 4 clusters
For epsilon = 9, the silhouette_score is 0.5405536726926806 and there are 3 clusters
For epsilon = 10, the silhouette_score is 0.539685763319866 and there are 3 clusters
For epsilon = 11, the silhouette_score is 0.5412603187776921 and there are 2 clusters
For epsilon = 12, the silhouette_score is 0.54126031877

In [414]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921}

In [415]:
best_eps

{'features1': [9, 10, 11], 'features2': [29], 'features3': [11, 12, 13]}

## Checking Out the 2 Cluster Epsilon

### NOTE: Uncovering Points That Aren't in NYC
While examing the 2 cluster epsilon values, I uncovered two points outside NYC that slipped through my cleaning. I went back to the features engineering data set (not having time to go back to see if I can fix the points at this stage in the work.)

The first time I ran these features, I found a point in Albany. Job no 420658176. Somehow it came in as being in Queens. It appears the longitude should have been 40.677346, not 42... . I need to drop this point in my data. Going back to clean up in feature engineering since I don't have time to go back and repair it. 

On rerunning with that point dropped, I found job_no 540197785, which is in Garden City, in Nassau County, not Queens.
After eliminating that, it appears that everything is truly within the NYC limits.

This work doesn't include those two points

-----

## Observations On The Two Clusters
Cluster 1 (42 observations) are all in historic districts. I believe that's the key differentiatior.

### Looking at averages for each cluster:
Compared to Cluster 0, Cluster 1 has/is:

- More subways nearby
- More parks nearby, with the closest park closer
- More criminal complaints
- Lower population.
- Higher household population rate, corresponding to lower group home rate.
- Lower rate of group home population (cluster 0: ~1.9%, cluster 2: ~1.5%)
- Higher rate of institutionalized people (though overall low percentage of population: cluster 0 ~.5%, cluster 1 ~.6%)
- Smaller average household size.
- Fewer people under 20.
- Racially, whiter, with more non-Hispanic multi-racial people.
- mostly in Brooklyn (47.6%), followed by Manhattan (34.7%), Queens (14.3%), and the Bronx (3.4%). There are no data points for permits in historic districts on Staten Island.

In [417]:
dbscan = DBSCAN(eps = 11)
dbscan.fit(X_scaled)
silhouette_score(X_scaled, dbscan.labels_)

0.5412603187776921

In [418]:
pd.Series(dbscan.labels_).unique()

array([0, 1])

In [419]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{0: 7135, 1: 42}

In [420]:
permits_X = X
permits_X['cluster']=dbscan.labels_
permits_X.head()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
0,121207354,40.798817,-73.968740,2255,0.033418,5,15,42,195,0.000000,2,17,33,214,9005,0.963,0.037,0.0,1.89,0.129,0.159,0.614,0.081,0.099,0.008,0.040,4951,0.924,0.076,0,1,0,0,0,0
1,121189524,40.860296,-73.926125,701,0.085621,2,15,34,153,0.049425,1,10,42,280,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0,0
2,121205445,40.858508,-73.929532,546,0.177308,0,6,35,146,0.115871,0,11,41,275,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0,0
3,220586168,40.818565,-73.918118,1578,0.167270,0,9,60,215,0.086285,1,32,93,418,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0,0
4,220593463,40.818742,-73.915997,2368,0.196656,0,12,61,216,0.134117,0,32,90,422,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0,0


In [421]:
permits_X.groupby('cluster').mean()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.862970e+08,40.674583,-73.960177,348.014436,0.853513,0.221584,5.111703,17.757954,93.444149,0.152742,0.722635,9.733987,30.681710,145.006727,4409.639103,0.980734,0.019266,0.005118,2.776685,0.206825,0.241184,0.389683,0.182258,0.141612,0.014405,0.030835,1735.548283,0.930296,0.069704,0.304975,0.037141,0.289839,0.265732,0.0
1,2.615742e+08,40.724009,-73.944212,636.238095,0.692310,0.476190,24.523810,73.476190,284.047619,0.086968,0.976190,18.023810,62.261905,279.428571,4252.023810,0.984976,0.015048,0.005619,2.120238,0.145667,0.117976,0.538714,0.169714,0.115571,0.008833,0.049190,2309.333333,0.901524,0.098476,0.476190,0.357143,0.142857,0.000000,1.0


In [268]:
permits_X.groupby(['cluster']).mean()

,job_no,latitude,longitude,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,closest_park,total_complaints,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3.862841e+08,40.674646,-73.960120,0.221739,5.112903,17.757083,93.422721,0.72216,9.727489,30.655820,144.993829,0.153080,347.704769,4408.991865,0.980731,0.019270,0.005121,2.776241,0.206753,0.241155,0.389680,0.182283,0.141616,0.014403,0.030841,1735.607574,0.930291,0.069709,0.305049,0.037167,0.289902,0.265638,0.0
1,2.615742e+08,40.724009,-73.944212,0.476190,24.523810,73.476190,284.047619,0.97619,18.023810,62.261905,279.428571,0.086968,636.238095,4252.023810,0.984976,0.015048,0.005619,2.120238,0.145667,0.117976,0.538714,0.169714,0.115571,0.008833,0.049190,2309.333333,0.901524,0.098476,0.476190,0.357143,0.142857,0.000000,1.0


## Checking Out the 3 Cluster Epsilon

In [422]:
dbscan = DBSCAN(eps = 9)
dbscan.fit(X_scaled)
silhouette_score(X_scaled, dbscan.labels_)

0.5405536726926806

In [423]:
pd.Series(dbscan.labels_).unique()

array([ 0,  1, -1])

In [424]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{-1: 3, 0: 7132, 1: 42}

In [425]:
permits_X = X
permits_X['cluster']=dbscan.labels_
permits_X.head()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
0,121207354,40.798817,-73.968740,2255,0.033418,5,15,42,195,0.000000,2,17,33,214,9005,0.963,0.037,0.0,1.89,0.129,0.159,0.614,0.081,0.099,0.008,0.040,4951,0.924,0.076,0,1,0,0,0,0
1,121189524,40.860296,-73.926125,701,0.085621,2,15,34,153,0.049425,1,10,42,280,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0,0
2,121205445,40.858508,-73.929532,546,0.177308,0,6,35,146,0.115871,0,11,41,275,7740,0.998,0.002,0.0,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0,0
3,220586168,40.818565,-73.918118,1578,0.167270,0,9,60,215,0.086285,1,32,93,418,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0,0
4,220593463,40.818742,-73.915997,2368,0.196656,0,12,61,216,0.134117,0,32,90,422,6893,0.979,0.021,0.0,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0,0


# Observations on the Clusters (2 Clusters and Outliers)
Cluster 1 is the same as for the 2 cluster epsilon. See observations above. 

Observations on Outliers.
2 of them (job nos 121185813 and 121191049) are extremely close to each other, possibly next door. Because they're so few and two of them are skewing the averages, I'm not going to look too closely.

Some observations. Outliers have:
- most/closest subways (by a significant scale)
- fewest closer parks, more parks than cluster 0 at 1 and 2.5 miles.
- furthest 'closest park'
- most criminal complaints by a large degree

In [426]:
permits_X.groupby('cluster').mean()

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
-1,2.213651e+08,40.735862,-73.923913,900.666667,0.029034,9.333333,56.666667,142.333333,335.333333,0.170995,0.333333,8.000000,32.666667,183.333333,3985.666667,0.752667,0.247333,0.000000,2.310000,0.115667,0.304667,0.309000,0.082000,0.221667,0.043333,0.040333,1581.666667,0.852667,0.147333,0.000000,0.666667,0.333333,0.000000,0.0
0,3.863664e+08,40.674557,-73.960192,347.781969,0.853860,0.217751,5.090017,17.705552,93.342400,0.152735,0.722799,9.734717,30.680875,144.990606,4409.817443,0.980830,0.019170,0.005120,2.776882,0.206863,0.241158,0.389717,0.182300,0.141579,0.014392,0.030831,1735.613012,0.930329,0.069671,0.305104,0.036876,0.289821,0.265844,0.0
1,2.615742e+08,40.724009,-73.944212,636.238095,0.692310,0.476190,24.523810,73.476190,284.047619,0.086968,0.976190,18.023810,62.261905,279.428571,4252.023810,0.984976,0.015048,0.005619,2.120238,0.145667,0.117976,0.538714,0.169714,0.115571,0.008833,0.049190,2309.333333,0.901524,0.098476,0.476190,0.357143,0.142857,0.000000,1.0


In [427]:
permits_X[permits_X['cluster'] == -1]

,job_no,latitude,longitude,total_complaints,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
369,421718485,40.712413,-73.783410,666,0.009898,16,16,24,55,0.065544,1,6,17,96,4173,0.996,0.004,0.0,3.45,0.203,0.206,0.035,0.096,0.471,0.114,0.079,1351,0.892,0.108,0,0,1,0,0,-1
6503,121185813,40.747544,-73.994063,1018,0.032286,6,77,202,476,0.229236,0,9,41,227,3892,0.631,0.369,0.0,1.74,0.072,0.354,0.446,0.075,0.097,0.008,0.021,1697,0.833,0.167,0,1,0,0,0,-1
6504,121191049,40.747629,-73.994265,1018,0.044918,6,77,201,475,0.218206,0,9,40,227,3892,0.631,0.369,0.0,1.74,0.072,0.354,0.446,0.075,0.097,0.008,0.021,1697,0.833,0.167,0,1,0,0,0,-1


-----

-----

# Removing Longitude and Latitude (aka Features 4)
This produces marginally better results.

In [555]:
to_drop4 = to_drop3 + ['latitude', 'longitude']
to_drop4

['bin_no',
 'house_no',
 'street_name',
 'zip_code',
 'job_start_date',
 'hist_dist_name',
 'name_closest_park',
 'closest_subway_line',
 'bct2020',
 'owners_business_type',
 'non-profit',
 'latitude',
 'longitude']

In [556]:
permits_for_testing = pd.read_csv('data/final_permits_for_testing.csv')

In [557]:
permits_for_testing.shape

(7177, 45)

In [558]:
perms_use4 = permits_for_testing.drop(columns = to_drop4)
perms_use4.shape

(7177, 32)

In [559]:
X = perms_use4
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [560]:
features = 'features4' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,12):
    for i in range(3, 6):
        dbscan = DBSCAN(eps = num, min_samples = i)
        dbscan.fit(X_scaled)
        score = silhouette_score(X_scaled, dbscan.labels_)
        pairs[num, i] = score
        scores.append(score)
        clusters = len(pd.Series(dbscan.labels_).unique())
        print(f"For epsilon = {num} and min samples {i}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_params_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_params[features] = best_params_list
print(best_params_list)

For epsilon = 1 and min samples 3, the silhouette_score is 0.23063270140057035 and there are 607 clusters
For epsilon = 1 and min samples 4, the silhouette_score is 0.12869390983883003 and there are 406 clusters
For epsilon = 1 and min samples 5, the silhouette_score is 0.05640263115947672 and there are 297 clusters
For epsilon = 2 and min samples 3, the silhouette_score is 0.0596530642250026 and there are 255 clusters
For epsilon = 2 and min samples 4, the silhouette_score is 0.04186956386541246 and there are 191 clusters
For epsilon = 2 and min samples 5, the silhouette_score is 0.026868678320988854 and there are 152 clusters
For epsilon = 3 and min samples 3, the silhouette_score is 0.008010785247039385 and there are 71 clusters
For epsilon = 3 and min samples 4, the silhouette_score is 0.007616736270278591 and there are 55 clusters
For epsilon = 3 and min samples 5, the silhouette_score is 0.013751172723738134 and there are 44 clusters
For epsilon = 4 and min samples 3, the silhoue

In [561]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609,
 'features11': 0.5025351034284963,
 'features12': 0.5335931852694467}

In [562]:
best_params

{'features5': [(9, 3),
  (9, 4),
  (9, 5),
  (10, 3),
  (10, 4),
  (10, 5),
  (11, 3),
  (11, 4),
  (11, 5),
  (12, 3),
  (12, 4),
  (12, 5),
  (13, 3),
  (13, 4),
  (13, 5)],
 'features4': [(11, 3), (11, 4), (11, 5)]}

# Trying Far Fewer Features (Feature List, aka Features 5)
Removing Census Data. In experimentation before final cleaning was complete, a variation onthis set of features got a silhouette score of ~.72. It included the items that I can't map onto my test data (i.e. ownership info, non profit status).

This was the original features list before modification (in case I want to use it above, again): 'features = ['subway_count_tenth_mi', 'subway_count_half_mi', 'subway_count_one_mi', 'subway_count_two_five_mi', 'park_count_tenth_mi', 'park_count_half_mi', 'park_count_one_mi', 'park_count_two_five_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN', 'borough_QUEENS', 'borough_STATEN ISLAND', 'owners_business_type_CORPORATION', 'owners_business_type_HPD', 'owners_business_type_INDIVIDUAL', 'owners_business_type_NYC AGENCY', 'owners_business_type_NYCHA/HHC', 'owners_business_type_OTHER', "owners_business_type_OTHER GOV'T AGENCY", 'owners_business_type_PARTNERSHIP', 'non-profit_Y', 'in_hist_dist_1']'

*The features below produce the best silhouette score for my test data to this point.*

In [549]:
features5 = ['closest_subway','subway_count_tenth_mi',
       'subway_count_half_mi', 'subway_count_one_mi',
       'subway_count_two_five_mi',
       'park_count_tenth_mi', 'park_count_half_mi', 'park_count_one_mi',
       'park_count_two_five_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND', 'in_hist_dist_1']

perms_use5 = permits_for_testing[features5]

perms_use5.shape

(7177, 16)

In [551]:
X = perms_use5
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features5' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,14):
    for i in range(3, 6):
        dbscan = DBSCAN(eps = num, min_samples = i)
        dbscan.fit(X_scaled)
        score = silhouette_score(X_scaled, dbscan.labels_)
        pairs[num, i] = score
        scores.append(score)
        clusters = len(pd.Series(dbscan.labels_).unique())
        print(f"For epsilon = {num} and min samples {i}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_params_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_params[features] = best_params_list
print(best_params_list)

For epsilon = 1 and min samples 3, the silhouette_score is -0.1096081924688761 and there are 130 clusters
For epsilon = 1 and min samples 4, the silhouette_score is -0.11466314727837579 and there are 85 clusters
For epsilon = 1 and min samples 5, the silhouette_score is -0.1136873054900191 and there are 59 clusters
For epsilon = 2 and min samples 3, the silhouette_score is 0.29717873547919826 and there are 21 clusters
For epsilon = 2 and min samples 4, the silhouette_score is 0.29693216950821905 and there are 17 clusters
For epsilon = 2 and min samples 5, the silhouette_score is 0.2950937398126008 and there are 15 clusters
For epsilon = 3 and min samples 3, the silhouette_score is 0.40506878810110813 and there are 11 clusters
For epsilon = 3 and min samples 4, the silhouette_score is 0.4173229373345217 and there are 6 clusters
For epsilon = 3 and min samples 5, the silhouette_score is 0.41751943503376127 and there are 6 clusters
For epsilon = 4 and min samples 3, the silhouette_score i

In [552]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609,
 'features11': 0.5025351034284963,
 'features12': 0.5335931852694467}

In [553]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9],
 'features9': [6, 7, 8, 9],
 'features10': [3],
 'features11': [6],
 'features12': [5]}

In [554]:
best_params

{'features5': [(9, 3),
  (9, 4),
  (9, 5),
  (10, 3),
  (10, 4),
  (10, 5),
  (11, 3),
  (11, 4),
  (11, 5),
  (12, 3),
  (12, 4),
  (12, 5),
  (13, 3),
  (13, 4),
  (13, 5)]}

## Checking Out Features 5, 2 Cluster Epsilon
I don't analyze here because I'm still honing features. Again, cluster 1 is historical distric permits.

In [451]:
dbscan = DBSCAN(eps = 9)
dbscan.fit(X_scaled)
silhouette_score(X_scaled, dbscan.labels_)

0.6838451018672402

In [452]:
pd.Series(dbscan.labels_).unique()

array([0, 1])

In [453]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{0: 7135, 1: 42}

In [454]:
permits_X = X
permits_X['cluster']=dbscan.labels_
permits_X.head()

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_1178/2589708616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_X['cluster']=dbscan.labels_


,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,closest_park,total_complaints,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
0,0.033418,5,15,42,195,2,17,33,214,0.000000,2255,0,1,0,0,0,0
1,0.085621,2,15,34,153,1,10,42,280,0.049425,701,0,1,0,0,0,0
2,0.177308,0,6,35,146,0,11,41,275,0.115871,546,0,1,0,0,0,0
3,0.167270,0,9,60,215,1,32,93,418,0.086285,1578,0,0,0,0,0,0
4,0.196656,0,12,61,216,0,32,90,422,0.134117,2368,0,0,0,0,0,0


In [455]:
permits_X.groupby('cluster').mean()

,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,closest_park,total_complaints,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,
0,0.853513,0.221584,5.111703,17.757954,93.444149,0.722635,9.733987,30.681710,145.006727,0.152742,348.014436,0.304975,0.037141,0.289839,0.265732,0.0
1,0.692310,0.476190,24.523810,73.476190,284.047619,0.976190,18.023810,62.261905,279.428571,0.086968,636.238095,0.476190,0.357143,0.142857,0.000000,1.0


## Checking Out Features 5, Clusters 3
Again, not analyzing in more depth because honing features further.

In [456]:
dbscan = DBSCAN(eps = 7)
dbscan.fit(X_scaled)
silhouette_score(X_scaled, dbscan.labels_)

0.6815658889923266

In [457]:
pd.Series(dbscan.labels_).unique()

array([ 0,  1, -1])

In [458]:
unique, counts = np.unique(dbscan.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{-1: 1, 0: 7134, 1: 42}

In [459]:
permits_X = X
permits_X['cluster']=dbscan.labels_
permits_X.head()

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_1178/2589708616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_X['cluster']=dbscan.labels_


,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,closest_park,total_complaints,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
0,0.033418,5,15,42,195,2,17,33,214,0.000000,2255,0,1,0,0,0,0
1,0.085621,2,15,34,153,1,10,42,280,0.049425,701,0,1,0,0,0,0
2,0.177308,0,6,35,146,0,11,41,275,0.115871,546,0,1,0,0,0,0
3,0.167270,0,9,60,215,1,32,93,418,0.086285,1578,0,0,0,0,0,0
4,0.196656,0,12,61,216,0,32,90,422,0.134117,2368,0,0,0,0,0,0


In [460]:
permits_X.groupby('cluster').mean()

,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,closest_park,total_complaints,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
cluster,,,,,,,,,,,,,,,,
-1,0.009898,16.000000,16.000000,24.000000,55.000000,1.000000,6.000000,17.000000,96.000000,0.065544,666.000000,0.000000,0.000000,1.000000,0.00000,0.0
0,0.853632,0.219372,5.110177,17.757079,93.449537,0.722596,9.734511,30.683628,145.013597,0.152754,347.969863,0.305018,0.037146,0.289739,0.26577,0.0
1,0.692310,0.476190,24.523810,73.476190,284.047619,0.976190,18.023810,62.261905,279.428571,0.086968,636.238095,0.476190,0.357143,0.142857,0.00000,1.0


In [461]:
permits_X[permits_X['cluster']==-1]

,closest_subway,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,closest_park,total_complaints,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
369,0.009898,16,16,24,55,1,6,17,96,0.065544,666,0,0,1,0,0,-1


# Features 6
I eliminated subways and parks 2.5 miles.

In [563]:
features6 = ['closest_subway', 'subway_count_tenth_mi',
       'subway_count_half_mi', 'subway_count_one_mi',
       'park_count_tenth_mi', 'park_count_half_mi', 'park_count_one_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND', 'in_hist_dist_1']

perms_use6 = permits_for_testing[features6]

perms_use6.shape

(7177, 14)

In [564]:
X = perms_use6
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features6' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,12):
    for i in range(3, 6):
        dbscan = DBSCAN(eps = num, min_samples = i)
        dbscan.fit(X_scaled)
        score = silhouette_score(X_scaled, dbscan.labels_)
        pairs[num, i] = score
        scores.append(score)
        clusters = len(pd.Series(dbscan.labels_).unique())
        print(f"For epsilon = {num} and min samples {i}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_params_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_params[features] = best_params_list
print(best_params_list)

For epsilon = 1 and min samples 3, the silhouette_score is -0.08173972462627385 and there are 95 clusters
For epsilon = 1 and min samples 4, the silhouette_score is -0.09477994023333916 and there are 62 clusters
For epsilon = 1 and min samples 5, the silhouette_score is -0.08013157425254705 and there are 48 clusters
For epsilon = 2 and min samples 3, the silhouette_score is 0.3250571490143033 and there are 19 clusters
For epsilon = 2 and min samples 4, the silhouette_score is 0.32867543687099204 and there are 14 clusters
For epsilon = 2 and min samples 5, the silhouette_score is 0.3355385183643135 and there are 11 clusters
For epsilon = 3 and min samples 3, the silhouette_score is 0.4395313269584088 and there are 11 clusters
For epsilon = 3 and min samples 4, the silhouette_score is 0.4510805074393287 and there are 6 clusters
For epsilon = 3 and min samples 5, the silhouette_score is 0.45079040533657916 and there are 6 clusters
For epsilon = 4 and min samples 3, the silhouette_score is

In [565]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609,
 'features11': 0.5025351034284963,
 'features12': 0.5335931852694467}

In [566]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9],
 'features9': [6, 7, 8, 9],
 'features10': [3],
 'features11': [6],
 'features12': [5]}

In [567]:
best_params

{'features5': [(9, 3),
  (9, 4),
  (9, 5),
  (10, 3),
  (10, 4),
  (10, 5),
  (11, 3),
  (11, 4),
  (11, 5),
  (12, 3),
  (12, 4),
  (12, 5),
  (13, 3),
  (13, 4),
  (13, 5)],
 'features4': [(11, 3), (11, 4), (11, 5)],
 'features6': [(9, 3),
  (9, 4),
  (9, 5),
  (10, 3),
  (10, 4),
  (10, 5),
  (11, 3),
  (11, 4),
  (11, 5)]}

# Features 7
I eliminated subways and parks 1 mile.

In [568]:
features7 = ['closest_subway', 'subway_count_tenth_mi',
       'subway_count_half_mi',
       'park_count_tenth_mi', 'park_count_half_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND', 'in_hist_dist_1']

perms_use7 = permits_for_testing[features7]

perms_use7.shape

(7177, 12)

In [570]:
X = perms_use7
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features7' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,11):
    for i in range(2, 6):
        dbscan = DBSCAN(eps = num, min_samples = i)
        dbscan.fit(X_scaled)
        score = silhouette_score(X_scaled, dbscan.labels_)
        pairs[num, i] = score
        scores.append(score)
        clusters = len(pd.Series(dbscan.labels_).unique())
        print(f"For epsilon = {num} and min samples {i}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_params_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_params[features] = best_params_list
print(best_params_list)

For epsilon = 1 and min samples 2, the silhouette_score is -0.056793147532545225 and there are 127 clusters
For epsilon = 1 and min samples 3, the silhouette_score is -0.011867843607873964 and there are 72 clusters
For epsilon = 1 and min samples 4, the silhouette_score is -0.014931893859834714 and there are 50 clusters
For epsilon = 1 and min samples 5, the silhouette_score is -0.02465166229988048 and there are 43 clusters
For epsilon = 2 and min samples 2, the silhouette_score is 0.3590644239293558 and there are 35 clusters
For epsilon = 2 and min samples 3, the silhouette_score is 0.36072126336812343 and there are 17 clusters
For epsilon = 2 and min samples 4, the silhouette_score is 0.3689669625888203 and there are 11 clusters
For epsilon = 2 and min samples 5, the silhouette_score is 0.3730504658065262 and there are 9 clusters
For epsilon = 3 and min samples 2, the silhouette_score is 0.47547789916400707 and there are 15 clusters
For epsilon = 3 and min samples 3, the silhouette_s

In [481]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414}

In [482]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11]}

# Features 8
I eliminated subways and parks 0.5 mile.

In [571]:
features8 = ['closest_subway', 'subway_count_tenth_mi',
       'park_count_tenth_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND', 'in_hist_dist_1']

perms_use8 = permits_for_testing[features8]

perms_use8.shape

(7177, 10)

In [573]:
X = perms_use8
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features8' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,7):
    for i in range(2, 6):
        dbscan = DBSCAN(eps = num, min_samples = i)
        dbscan.fit(X_scaled)
        score = silhouette_score(X_scaled, dbscan.labels_)
        pairs[num, i] = score
        scores.append(score)
        clusters = len(pd.Series(dbscan.labels_).unique())
        print(f"For epsilon = {num} and min samples {i}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_params_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_params[features] = best_params_list
print(best_params_list)

For epsilon = 1 and min samples 2, the silhouette_score is 0.12446057625794153 and there are 71 clusters
For epsilon = 1 and min samples 3, the silhouette_score is 0.12324986642025955 and there are 47 clusters
For epsilon = 1 and min samples 4, the silhouette_score is 0.1257804613007205 and there are 38 clusters
For epsilon = 1 and min samples 5, the silhouette_score is 0.12139125404608181 and there are 31 clusters
For epsilon = 2 and min samples 2, the silhouette_score is 0.423930492502996 and there are 22 clusters
For epsilon = 2 and min samples 3, the silhouette_score is 0.4237491609592356 and there are 17 clusters
For epsilon = 2 and min samples 4, the silhouette_score is 0.42784193328018655 and there are 12 clusters
For epsilon = 2 and min samples 5, the silhouette_score is 0.4300335015485082 and there are 11 clusters
For epsilon = 3 and min samples 2, the silhouette_score is 0.5129662748361689 and there are 9 clusters
For epsilon = 3 and min samples 3, the silhouette_score is 0.5

In [489]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803}

In [490]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9]}

# Features 9
I eliminated subways and parks 0.1 mile.

In [597]:
features9 = ['closest_subway', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND', 'in_hist_dist_1']

perms_use9 = permits_for_testing[features9]

perms_use9.shape

(7177, 8)

In [606]:
X = perms_use9
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

In [575]:
features = 'features9' #CHANGE THIS
pairs = {}
scores = []
for num in range(1,10):
    for i in range(2, 7):
        dbscan = DBSCAN(eps = num, min_samples = i)
        dbscan.fit(X_scaled)
        score = silhouette_score(X_scaled, dbscan.labels_)
        pairs[num, i] = score
        scores.append(score)
        clusters = len(pd.Series(dbscan.labels_).unique())
        print(f"For epsilon = {num} and min samples {i}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_params_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_params[features] = best_params_list
print(best_params_list)
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 1 and min samples 2, the silhouette_score is 0.5159967952796181 and there are 13 clusters
For epsilon = 1 and min samples 3, the silhouette_score is 0.5159967952796181 and there are 13 clusters
For epsilon = 1 and min samples 4, the silhouette_score is 0.5170170737402451 and there are 12 clusters
For epsilon = 1 and min samples 5, the silhouette_score is 0.5233515801208046 and there are 11 clusters
For epsilon = 1 and min samples 6, the silhouette_score is 0.5371995402650572 and there are 9 clusters
For epsilon = 2 and min samples 2, the silhouette_score is 0.5374664966138338 and there are 11 clusters
For epsilon = 2 and min samples 3, the silhouette_score is 0.5378486762573811 and there are 10 clusters
For epsilon = 2 and min samples 4, the silhouette_score is 0.5378486762573811 and there are 10 clusters
For epsilon = 2 and min samples 5, the silhouette_score is 0.5377646543369198 and there are 10 clusters
For epsilon = 2 and min samples 6, the silhouette_score is 0.5369

In [576]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609,
 'features11': 0.5025351034284963,
 'features12': 0.5335931852694467}

In [521]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9],
 'features9': [6, 7, 8, 9],
 'features10': [3],
 'features11': [6],
 'features12': [5]}

## Making Model For Use In Function

In [525]:
dbscan_model = DBSCAN(eps = 6).fit(X_scaled)

In [526]:
silhouette_score(X_scaled, dbscan_model.labels_)

0.7622861632347829

I've tried this pair (eps = 2 min_samples = 2) because it gives lots of clusters without losing too much silhouette score (compared to other models that don't produce 2 clusters). The low epsilon also requires being closer to an existing cluster than otherwise.

In [577]:
dbscan_model2 = DBSCAN(eps = 2, min_samples = 2).fit(X_scaled)

In [578]:
silhouette_score(X_scaled, dbscan_model2.labels_)

0.5374664966138338

In [583]:
pd.Series(dbscan_model2.labels_).unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, -1])

In [584]:
unique, counts = np.unique(dbscan_model2.labels_, return_counts=True)

count = dict(zip(unique, counts))
count

{-1: 3,
 0: 265,
 1: 728,
 2: 2175,
 3: 1896,
 4: 1918,
 5: 20,
 6: 15,
 7: 5,
 8: 150,
 9: 2}

In [585]:
permits_X = X
permits_X['cluster']=dbscan_model2.labels_
permits_X.head()

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_1178/594924473.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  permits_X['cluster']=dbscan_model2.labels_


,closest_subway,closest_park,total_complaints,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1,cluster
0,0.033418,0.000000,2255,0,1,0,0,0,0
1,0.085621,0.049425,701,0,1,0,0,0,0
2,0.177308,0.115871,546,0,1,0,0,0,0
3,0.167270,0.086285,1578,0,0,0,0,0,1
4,0.196656,0.134117,2368,0,0,0,0,0,1


## Prepping Test Data

In [588]:
test.head(1)

,longitude,latitude,total_complaints,closest_subway,closest_subway_line,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,hist_dist_name,closest_park,name_closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,bct2020,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
0,-73.739823,40.597312,271.0,0.959344,A,0.0,0.0,1.0,12.0,none,0.0,Seagirt Ave Wetlands,1.0,5.0,10.0,29.0,4101002,"4,596",1.0,0.0,0.0,2.71,0.321,0.072,0.709,0.174,0.005,0.019,0.022,"1,811",0.937,0.063,0,0,1,0,0


In [589]:
test.shape

(105, 37)

In [590]:
features9 = ['closest_subway', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND', 'in_hist_dist_1']

test9 = test[features9]

test9.shape

(105, 8)

In [600]:
X = test9
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

# Function To See If New Data Fits Clusters DOESN'T APPEAR TO BE WORKING.
Using code from [this stackoverflow answer](https://stackoverflow.com/a/35458920/5394724) to see if my new points fit in these clusters or not.

This compares the new element to the core.

In [543]:
def dbscan_predict(dbscan_model, X_new, metric=sp.spatial.distance.cosine):
    # Result is noise by default
    y_new = np.ones(shape=len(X_new), dtype=int)*-1 

    # Iterate all input samples for a label
    for j, x_new in enumerate(X_new):
        # Find a core sample closer than EPS
        for i, x_core in enumerate(dbscan_model.components_): 
            if metric(x_new, x_core) < dbscan_model.eps:
                # Assign label of x_core to x_new
                y_new[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
                break

    return y_new

## Making Predictions

In [601]:
dbscan_predict(dbscan_model, X_scaled)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [602]:
predictions2 = dbscan_predict(dbscan_model2, X_scaled)

In [603]:
predictions2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [611]:
len(X_scaled[4])

8

In [610]:
len(X_scaled)

7177

In [609]:
dbscan_predict(dbscan_model2, X_scaled[4])

array([0, 0, 0, 0, 0, 0, 0, 0])

# Features 10
I eliminated historic districts

In [496]:
features10 = ['closest_subway', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND']

perms_use10 = permits_for_testing[features10]

perms_use10.shape

(7177, 7)

In [498]:
X = perms_use10
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features10' #CHANGE THIS
pairs = {}
scores = []
for num in range(1, 6):
    dbscan = DBSCAN(eps = num)
    dbscan.fit(X_scaled)
    score = silhouette_score(X_scaled, dbscan.labels_)
    pairs[num] = score
    scores.append(score)
    clusters = len(pd.Series(dbscan.labels_).unique())
    print(f"For epsilon = {num}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 1, the silhouette_score is 0.5223999122018389 and there are 8 clusters
For epsilon = 2, the silhouette_score is 0.5363796143217333 and there are 7 clusters
For epsilon = 3, the silhouette_score is 0.5439782055637609 and there are 4 clusters
For epsilon = 4, the silhouette_score is 0.5265868294879232 and there are 2 clusters
For epsilon = 5, the silhouette_score is 0.5265868294879232 and there are 2 clusters
[3]


In [499]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609}

In [500]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9],
 'features9': [6, 7, 8, 9],
 'features10': [3]}

# Features 11
Trying restoring census data.This confirms that the census data doesn't help the clustering

In [504]:
to_use = ['bct2020', 'pop_20', 'hhpop_20p', 'gq_20p', 'instgq_20p', 'avhhsz_20', 'popu18_20p', 'hsp_20p', 'wnh_20p', 'bnh_20p', 'anh_20p', 'onh_20p', 'nh2pl_20p', 'hunits_20', 'ochu_20p', 'vachu_20p']


In [505]:
features11 = ['closest_subway', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND']+to_use

perms_use11 = permits_for_testing[features11]

perms_use11.shape

(7177, 23)

In [ ]:
X = perms_use11
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features11' #CHANGE THIS
pairs = {}
scores = []
for num in range(1, 9):
    dbscan = DBSCAN(eps = num)
    dbscan.fit(X_scaled)
    score = silhouette_score(X_scaled, dbscan.labels_)
    pairs[num] = score
    scores.append(score)
    clusters = len(pd.Series(dbscan.labels_).unique())
    print(f"For epsilon = {num}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 1, the silhouette_score is 0.23059840119914968 and there are 353 clusters
For epsilon = 2, the silhouette_score is 0.05945258005864663 and there are 91 clusters
For epsilon = 3, the silhouette_score is 0.18953860265516073 and there are 22 clusters
For epsilon = 4, the silhouette_score is 0.3441466196774612 and there are 7 clusters
For epsilon = 5, the silhouette_score is 0.37539368015327695 and there are 5 clusters
For epsilon = 6, the silhouette_score is 0.5025351034284963 and there are 3 clusters
For epsilon = 7, the silhouette_score is 0.5021588000354729 and there are 3 clusters
For epsilon = 8, the silhouette_score is 0.5019920229550395 and there are 2 clusters
[6]


In [ ]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609,
 'features11': 0.5025351034284963}

In [ ]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9],
 'features9': [6, 7, 8, 9],
 'features10': [3],
 'features11': [6]}

# Features 12
No census data. Adding in some subway and park data without restoring historic districts to see if they help. Going for .5 mile to start.

In [511]:
permits_for_testing

,bin_no,house_no,street_name,job_no,zip_code,job_start_date,owners_business_type,non-profit,latitude,longitude,bct2020,total_complaints,closest_subway,closest_subway_line,subway_count_tenth_mi,subway_count_half_mi,subway_count_one_mi,subway_count_two_five_mi,hist_dist_name,closest_park,name_closest_park,park_count_tenth_mi,park_count_half_mi,park_count_one_mi,park_count_two_five_mi,pop_20,hhpop_20p,gq_20p,instgq_20p,avhhsz_20,popu18_20p,hsp_20p,wnh_20p,bnh_20p,anh_20p,onh_20p,nh2pl_20p,hunits_20,ochu_20p,vachu_20p,borough_BROOKLYN,borough_MANHATTAN,borough_QUEENS,borough_STATEN ISLAND,in_hist_dist_1
0,1056547,2686,BROADWAY,121207354,10025,2022-05-11,PARTNERSHIP,N,40.798817,-73.968740,1019100,2255,0.033418,1,5,15,42,195,none,0.000000,Broadway Malls,2,17,33,214,9005,0.963,0.037,0.00,1.89,0.129,0.159,0.614,0.081,0.099,0.008,0.040,4951,0.924,0.076,0,1,0,0,0
1,1812187,140,HILLSIDE AVENUE,121189524,10040,2022-05-11,CORPORATION,N,40.860296,-73.926125,1028300,701,0.085621,1,2,15,34,153,none,0.049425,Highbridge Park,1,10,42,280,7740,0.998,0.002,0.00,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0
2,1091669,37,HILLSIDE AVENUE,121205445,10040,2021-02-19,PARTNERSHIP,N,40.858508,-73.929532,1028300,546,0.177308,A,0,6,35,146,none,0.115871,Fort Tryon Park,0,11,41,275,7740,0.998,0.002,0.00,2.30,0.153,0.673,0.189,0.066,0.041,0.009,0.022,3517,0.957,0.043,0,1,0,0,0
3,2823631,368,EAST 152 STREET,220586168,10455,2022-05-11,INDIVIDUAL,N,40.818565,-73.918118,2006700,1578,0.167270,2-5,0,9,60,215,none,0.086285,Melrose Playground,1,32,93,418,6893,0.979,0.021,0.00,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0
4,2129278,418,EAST 153 STREET,220593463,10455,2018-02-09,CORPORATION,N,40.818742,-73.915997,2006700,2368,0.196656,2-5,0,12,61,216,none,0.134117,P.S. 29 Ballfield,0,32,90,422,6893,0.979,0.021,0.00,2.85,0.264,0.710,0.017,0.245,0.008,0.010,0.011,2439,0.970,0.030,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7172,3171719,1266,OCEAN PARKWAY,321419826,11230,2018-06-05,CORPORATION,N,40.617918,-73.969296,3044800,139,0.303992,F,0,7,28,128,none,0.006314,Ocean Parkway Malls,1,4,19,112,1965,1.000,0.000,0.00,2.54,0.262,0.079,0.740,0.021,0.105,0.020,0.034,841,0.920,0.080,1,0,0,0,0
7173,3038424,698,EMPIRE BLVD,321594164,11213,2019-12-13,CORPORATION,N,40.663843,-73.938947,3033100,436,0.428943,3,0,4,20,177,none,0.083542,Hamilton Metz Field,1,3,29,277,4217,0.999,0.001,0.00,2.87,0.281,0.042,0.587,0.289,0.002,0.044,0.036,1531,0.958,0.042,1,0,0,0,0
7174,4607406,155-100,BAYVIEW AVENUE,420653634,11422,2017-03-07,INDIVIDUAL,N,40.636722,-73.746498,4066401,13,2.378776,A,0,0,0,1,none,0.188279,Idlewild Park,0,2,3,38,1923,0.888,0.112,0.00,3.18,0.168,0.148,0.051,0.709,0.016,0.019,0.056,574,0.936,0.064,0,0,1,0,0
7175,4620655,44-19,64 STREET,420661965,11377,2019-10-01,INDIVIDUAL,N,40.740692,-73.901679,4024500,290,0.363698,7,0,4,27,125,none,0.022918,Big Bush Playground,1,12,38,177,5544,0.986,0.014,0.00,2.51,0.153,0.322,0.310,0.031,0.304,0.009,0.025,2262,0.963,0.037,0,0,1,0,0


In [512]:
features12 = ['closest_subway', 'subway_count_half_mi', 'closest_park', 'park_count_half_mi', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND']

perms_use12 = permits_for_testing[features12]

perms_use12.shape

(7177, 9)

In [514]:
X = perms_use12
ss = StandardScaler()
X_scaled = ss.fit_transform(X)

features = 'features12' #CHANGE THIS
pairs = {}
scores = []
for num in range(1, 6):
    dbscan = DBSCAN(eps = num)
    dbscan.fit(X_scaled)
    score = silhouette_score(X_scaled, dbscan.labels_)
    pairs[num] = score
    scores.append(score)
    clusters = len(pd.Series(dbscan.labels_).unique())
    print(f"For epsilon = {num}, the silhouette_score is {score} and there are {clusters} clusters")
max_score = max(scores)
best_eps_list = [i for i in pairs if pairs[i] == max_score]
best_scores[features] = max_score
best_eps[features] = best_eps_list
print(best_eps_list)

For epsilon = 1, the silhouette_score is 0.38370768908929537 and there are 12 clusters
For epsilon = 2, the silhouette_score is 0.4375674361902203 and there are 7 clusters
For epsilon = 3, the silhouette_score is 0.5174397847552017 and there are 4 clusters
For epsilon = 4, the silhouette_score is 0.5328223597869812 and there are 3 clusters
For epsilon = 5, the silhouette_score is 0.5335931852694467 and there are 2 clusters
[5]


In [515]:
best_scores

{'features1': 0.5631723159341483,
 'features2': 0.8598814471096117,
 'features3': 0.5412603187776921,
 'features4': 0.5559491055735374,
 'features5': 0.6838451018672402,
 'features6': 0.700882156558051,
 'features7': 0.714928897943414,
 'features8': 0.7326281644636803,
 'features9': 0.7622861632347829,
 'features10': 0.5439782055637609,
 'features11': 0.5025351034284963,
 'features12': 0.5335931852694467}

In [516]:
best_eps

{'features1': [9, 10, 11],
 'features2': [29],
 'features3': [11, 12, 13],
 'features4': [11],
 'features5': [9, 10, 11],
 'features6': [9, 10, 11],
 'features7': [9, 10, 11],
 'features8': [6, 7, 8, 9],
 'features9': [6, 7, 8, 9],
 'features10': [3],
 'features11': [6],
 'features12': [5]}

-----

# NOTE: THESE ARE FOR THE ORIGINAL FEATURE LIST

# Notes on The Clusters
One cluster is just the ones in the historic districts (cluster 1). Cluster 2 is very far from parks, and is in Queens. Cluster 1 is all in the Bronx and is far from parks.
## I think I have some outliers I need to remove. Based on map, yesterday. Look at Tableau.
I could do a check to make sure the point falls within the boundaries of the borough, too.

# Include Longitude and Latitude

In [301]:
features2 = ['latitude', 'longitude','closest_subway', 'subway_count_tenth_mi',
       'subway_count_half_mi', 'subway_count_one_mi',
       'subway_count_two_five_mi',
       'park_count_tenth_mi', 'park_count_half_mi', 'park_count_one_mi',
       'park_count_two_five_mi', 'closest_park', 'total_complaints', 'borough_BROOKLYN', 'borough_MANHATTAN',
       'borough_QUEENS', 'borough_STATEN ISLAND',
       'owners_business_type_CORPORATION', 'owners_business_type_HPD',
       'owners_business_type_INDIVIDUAL', 'owners_business_type_NYC AGENCY',
       'owners_business_type_NYCHA/HHC', 'owners_business_type_OTHER',
       "owners_business_type_OTHER GOV'T AGENCY",
       'owners_business_type_PARTNERSHIP', 'non-profit_Y', 'in_hist_dist_1']

In [302]:
X2 = permits[features2]

KeyError: "['closest_subway'] not in index"

In [166]:
ss = StandardScaler()
X2_scaled = ss.fit_transform(X2)

In [167]:
dbscan2 = DBSCAN(eps=1)
dbscan2.fit(X2_scaled)

DBSCAN(eps=1)

In [168]:
pd.Series(dbscan2.labels_).unique()

array([-1,  0,  1,  2,  3,  4,  5,  6, 21,  8,  7,  9, 10, 11, 12, 42, 13,
       14, 15, 16, 17, 18, 19, 20, 60, 22, 23, 24, 53, 54, 25, 26, 58, 61,
       39, 27, 28, 62, 29, 59, 49, 30, 33, 31, 32, 34, 43, 35, 36, 37, 38,
       65, 45, 51, 40, 41, 44, 52, 46, 47, 48, 50, 55, 56, 57, 63, 64])

In [169]:
silhouette_score(X2_scaled, dbscan2.labels_)

0.03985773625180413

In [170]:
dbscan2 = DBSCAN(eps=10)
dbscan2.fit(X2_scaled)

DBSCAN(eps=10)

In [171]:
pd.Series(dbscan2.labels_).unique()

array([ 0,  1,  2,  3, -1])

In [172]:
silhouette_score(X2_scaled, dbscan2.labels_)

0.6295941401638957

In [173]:
dbscan2 = DBSCAN(eps=15)
dbscan2.fit(X2_scaled)

DBSCAN(eps=15)

In [174]:
pd.Series(dbscan2.labels_).unique()

array([ 0,  1,  2, -1])

In [175]:
silhouette_score(X2_scaled, dbscan2.labels_)

0.7005983949467619

This is a better fit without longitude and latitude